Import required modules, libraries and Shakespearean text for project

In [80]:
from __future__ import print_function
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.tokenize import TreebankWordTokenizer
import urllib.request
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from textblob import TextBlob
from nltk.util import ngrams
from collections import Counter
from operator import itemgetter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse.linalg import svds
import gensim
from functools import reduce
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import pandas as pd
import os
import string
import re
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

Much_Ado = urllib.request.urlopen('http://www.textfiles.com/etext/AUTHORS/SHAKESPEARE/shakespeare-much-3.txt').read()
Much_Ado_lines = Much_Ado.decode('utf8')
Tempest = urllib.request.urlopen('http://www.textfiles.com/etext/AUTHORS/SHAKESPEARE/shakespeare-tempest-4.txt').read()
Tempest_lines = Tempest.decode('utf8')
Richard = urllib.request.urlopen('http://www.textfiles.com/etext/AUTHORS/SHAKESPEARE/shakespeare-tragedy-58.txt').read()
Richard_lines = Richard.decode('utf8')


Introduce stopwords, including a corpus of Shakespearean stopwords.

In [100]:
stop = stopwords.words('english')
stop += ['|','[',']','And','Because','S','For','An','To','From','But','That','If','For','To','As'
        'Than','Messenger','shall','must']
stop += open('new_stop.txt','r').readlines()
stop = [i.replace('\n',"") for i in stop]



Create functions to clean text and separate by act

In [98]:
def MySentences(Object):
    vocab = []
    lemmatizer = nltk.stem.WordNetLemmatizer()
    Object = lemmatizer.lemmatize(Object[Object.find("ACT"):])
    for line in sent_tokenize(Object):
        line = re.sub('[A-Z]{2,}',' ',line)
        line = re.sub("\[[^\]]*\]","",line)
        line= re.sub('[%s]' % re.escape(string.punctuation), '', line)
        vocab.append(line.split())
    vocab_2 = [] 
    for i in vocab:
        vocab_2.append(list(filter(lambda x: x not in stop, i)))
    return vocab_2

def Find_Acts(play):
    play_Acts = {}
    play_Acts2 = {}
    i = 0
    b = 0
    for x in range(1,play.count('ACT')+1):
        i = play.find("ACT",+ b)
        b = play.find("ACT", i + 1)
        play_Acts[x] = play[i:b]
    for i in ['ACT I','ACT II','ACT III','ACT IV','ACT V']:
        play_Acts2[i] = []
    for k,v in play_Acts.items():
        for i,j in play_Acts2.items():
            if i in v.split('\n'):
                play_Acts2[i].append(v)
    return play_Acts2

In [103]:
def gensim_model(play,Char):
    most_similar_list = []
    Act_list = ['ACT I','ACT II','ACT III','ACT IV','ACT V']
    for i in Act_list:
        try:
            model = gensim.models.Word2Vec(MySentences(''.join(Find_Acts(play)[i])),size=100,min_count=1,workers=2)
            most_similar_list.append((Char,i,[x[0] for x in model.most_similar(Char,topn=20)],
                                 TextBlob(' '.join([x[0] for x in model.most_similar(Char,topn=20)])).sentiment[0]))
        except: 
            most_similar_list.append((Char,i,[],0))
    return most_similar_list
print(gensim_model(Richard_lines,'Anne'))

/Users/jitsen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys
/Users/jitsen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('Anne', 'ACT I', ['sister', 'Lo', 'presence', 'bottled', 'sin', 'vassal', 'snow', 'worship', 'mind', 'threat', 'imprisond', 'could', 'breaks', 'smoothing', 'suspect', 'lightly', 'wellspoken', 'reflecting', 'obstacles', 'forgive'], 0.17500000000000002), ('Anne', 'ACT II', [], 0), ('Anne', 'ACT III', [], 0), ('Anne', 'ACT IV', ['sister', 'Lo', 'tonguetied', 'bottled', 'sin', 'step', 'kingly', 'mind', 'could', 'pluck', 'ah', 'Whisper', 'benefit', 'haughty', 'George', 'A', 'mad', 'lets', 'revenge', 'blunt'], -0.625), ('Anne', 'ACT V', ['Lo', 'horses', 'shock', 'midst', 'snow', 'threat', 'mind', 'could', 'breaks', 'alacrity', 'George', 'A', 'mad', 'mount', 'fatherinlaw', 'frownd', 'revenge', 'oer', 'trembling', 'led'], -0.625)]
